In [4]:
import pandas as pd
import great_expectations as gx
from zipfile import ZipFile
import requests
from pathlib import Path

In [2]:
!curl -Ls "https://info.stackoverflowsolutions.com/rs/719-EMH-566/images/stack-overflow-developer-survey-2022.zip" -o stack-overflow-developer-survey-2022.zip


# https://survey.stackoverflow.co/datasets/stack-overflow-developer-survey-2017.zip

^C


In [36]:

def extract(year):
    resp = requests.get(f"https://survey.stackoverflow.co/datasets/stack-overflow-developer-survey-{year}.zip")
    data_dir = Path(f'./stack_data/{year}')
    data_dir.mkdir(exist_ok=True, parents=True)
    with open(data_dir / f'stack-overflow-developer-survey-{year}.zip', 'wb') as f:
        f.write(resp.content)
    return data_dir / f'stack-overflow-developer-survey-{year}.zip'

def transform(file_path: Path):
    zip = ZipFile(file_path)
    for file in zip.infolist():
        if  Path(file.filename).is_file():
            print(file.orig_filename)
            zip.extract(file.filename, path=file_path.parent)
# def dq_checks():
# def load():

def orchestrate(start_year, end_year):
    for year in range(start_year, end_year + 1):
        file_path = extract(year)
        print(file_path)
        transform(file_path)

if __name__ == '__main__':
    orchestrate(2011, 2015)

stack_data/2011/stack-overflow-developer-survey-2011.zip
stack_data/2012/stack-overflow-developer-survey-2012.zip
stack_data/2013/stack-overflow-developer-survey-2013.zip
stack_data/2014/stack-overflow-developer-survey-2014.zip
stack_data/2015/stack-overflow-developer-survey-2015.zip
